# Data Aggregation

## HTML Parsing

Read the HTML source from the exam schedule webpage:

In [43]:
import requests

url = 'http://registrar.emory.edu/faculty-staff/exam-schedule/spring-2018.html'
r = requests.get(url)
print(r.text[:85])

<!DOCTYPE html><html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en">
  


Parse the source to retrieve the exam schedule:

In [18]:
from bs4 import BeautifulSoup

html = BeautifulSoup(r.text, 'html.parser')
tbody = html.find('tbody')
exam_schedule = {}

for tr in tbody.find_all('tr'):
    tds  = tr.find_all('td')
    meet = tds[0].string
    day  = tds[1].string
    date = tds[2].string
    time = tds[3].string
    exam_schedule[meet] = (day, date, time)

for k, v in exam_schedule.items():
    print('%10s: %s' % (k, str(v)))

 08:00 MWF: ('Tuesday', '8-May', '11:30 A.M - 2:00 P.M')
08:00 TTHF: ('Tuesday', '8-May', '3:00 P.M - 5:30 P.M')
  08:30 MW: ('Tuesday', '8-May', '11:30 A.M - 2:00 P.M')
 08:30 TTH: ('Tuesday', '8-May', '3:00 P.M - 5:30 P.M')
 09:00 MWF: ('Tuesday', '8-May', '11:30 A.M - 2:00 P.M')
09:00 TThF: ('Tuesday', '8-May', '3:00 P.M - 5:30 P.M')
 10:00 MW : ('Monday', '7-May', '8:00 A.M - 10:30 A.M')
 10:00 MWF: ('Monday', '7-May', '8:00 A.M - 10:30 A.M')
10:00 TTH : ('Tuesday', '8-May', '8:00 A.M - 10:30 A.M')
10:00 TThF: ('Tuesday', '8-May', '8:00 A.M - 10:30 A.M')
11:00 MWFf: ('Friday', '4-May', '8:00 A.M - 10:30 A.M')
11:00 TThF: ('Thursday', '3-May', '8:00 A.M - 10:30 A.M')
  11:30 MW: ('Friday', '4-May', '8:00 A.M - 10:30 A.M')
 11:30 TTh: ('Thursday', '3-May', '8:00 A.M - 10:30 A.M')
 12:00 MWF: ('Friday', '4-May', '3:00 P.M - 5:30 P.M')
12:00 TThF: ('Friday', '4-May', '3:00 P.M - 5:30 P.M')
   1:00 MW: ('Thursday', '3-May', '3:00 P.M - 5:30 P.M')
  1:00 MWF: ('Thursday', '3-May', '3:00 

## Regular Expressions

Retrieve rows selectively:

In [23]:
import re

TIME = re.compile('(\d{1,2}:\d\d)\s+([A-Za-z]+)')
m = TIME.match('8:00 MW')
print('Time: %s, Day: %s' % (m.group(1), m.group(2)))
m = TIME.match('12:30 TThF')
print('Time: %s, Day: %s' % (m.group(1), m.group(2)))

Time: 8:00, Day: MW
Time: 12:30, Day: TThF


In [27]:
m = TIME.match('Math*')
print(m)

None
